# Spatial selection with tripolar ACCESS-OM2 grid

The ACCESS-OM2 model collection utilises a tripolar grid north of 65N to avoid a point of convergence over the ocean at the north pole.

This means any plotting or analysis in this region needs to use 2D curvilinear latitude and longitude coordinates.

This notebook will cover how to use the curvilinear coordinates to select data within latitude and longitude limits, and how to plot data in the tripole region correctly. There is also a [full tutorial on plotting with projections](https://cosima-recipes.readthedocs.io/en/latest/Tutorials/Making_Maps_with_Cartopy.html).

Below the tripole area it is sufficient to use 1D latitude and longitude coordinates, which are much more convenient to use as they can use the [xarray](http://xarray.pydata.org/en/stable/index.html)'s [sel](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.sel.html) method with `slice` notation.

This notebook will also demonstrate how to do this.

The data output from the CICE model component do not have a convenient to use 1D latitude and longitude coordinates. As the ice and ocean models share a grid the coordinates from each model can be used interchangeably with the other. A method to add 1D latitude and longitude coordinates to an ice [`xarray.DataArray`](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html) is also demonstrated.

In [1]:
import intake
catalog = intake.cat.access_nri
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import cartopy.crs as ccrs

## Load data

Open a database session

In [2]:
from dask.distributed import Client
client = Client("tcp://10.6.77.34:8786")
client

<Client: 'tcp://10.6.77.34:8786' processes=1 threads=12, memory=46.00 GiB>

The specific experiment does not matter a great deal. This is the main IAF control run with JRA55 v1.4 forcing

In [3]:
expt = '01deg_jra55v140_iaf'

Load the 1D latitude/longitude variables from the ocean data to use them with ice data. The dimensions are renamed to match the required dimension in the ice data

In [4]:
cat_subset = catalog[expt]
var_search = cat_subset.search(variable='yt_ocean')
var_search = var_search.search(path=var_search.df['path'][0])
darray = var_search.to_dask()
darray = darray['yt_ocean']
yt_ocean = darray.rename({'yt_ocean': 'latitude'})
cat_subset = catalog[expt]
var_search = cat_subset.search(variable='xt_ocean')
var_search = var_search.search(path=var_search.df['path'][0])
darray = var_search.to_dask()
darray = darray['xt_ocean']
xt_ocean = darray.rename({'xt_ocean': 'longitude'})

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)


KeyError: 0

Both the ocean and the ice output variables have masked curvilinear coordinates, which means there are missing values for the coordinates over land. These variables are not suitable for plotting with [cartopy](https://scitools.org.uk/cartopy/docs/latest/). A full curvilinear grid is available from the CICE model input grid:

In [ ]:
ice_grid = xr.open_dataset('/g/data/ik11/inputs/access-om2/input_eee21b65/cice_01deg/grid.nc')

Extract the full curvilinear coordinates, from the ice grid. Note that these are in radians and so neeed to converted to degrees, and again the dimensions are renamed to match the dataset to which they will be added:

In [ ]:
geolon_t = np.degrees(ice_grid.tlon.rename({'nx': 'longitude', 'ny': 'latitude'}))
geolat_t = np.degrees(ice_grid.tlat.rename({'nx': 'longitude', 'ny': 'latitude'}))

Load an ice variable to use as an example dataset. In this case only the first 12 months for purposes of illustration.

In [ ]:
cat_subset = catalog[expt]
var_search = cat_subset.search(variable='aice_m')
darray = var_search.to_dask()
darray = darray['aice_m']
aice_m = darray

Printing the variable shows it has non-informative dimension names and no CF coordinate variables. As a result when the first time slice is plotted there is no useful units automatically added to the plot. 

In [ ]:
aice_m

Define colormap for plotting. There are two options here depending on your preference.
1. Option 1: Standard `ice` colormap from cmocean
2. Option 2: Colormap from dark blue (open ocean) to white (100% sea ice cover)

In [ ]:
def ice_cmap(colormap):
    from palettable.colorbrewer.sequential import Blues_9_r
    import cmocean as cmo

    if colormap == 'Option 1':
        cmap = cmo.cm.ice
        cmap.set_bad('lightgrey') # set color for land
    else:
        cmap = Blues_9_r.mpl_colormap
        cmap.set_bad('lightgrey') # set color for land

    return cmap

In [ ]:
colormap = 'Option 2'
cmap = ice_cmap(colormap)
cmap

In [ ]:
aice_m.isel(time=0).plot(aspect=2,size=10, cmap=cmap);

## Add spatial coordinates

To improve usability CF compatible spatial coordinates from the ocean data loaded above can be added to the ice data.

The unintuitive dimensions are renamed, and importantly match the dimension names chosen for the coordinates. The coordinates loaded above are added to the `DataArray`, and saved in a new variable, `aice_m_coords`.

In [ ]:
aice_m_coords = aice_m.rename({'nj':'latitude', 
                               'ni':'longitude'}).assign_coords({'latitude': yt_ocean, 
                                                                 'longitude': xt_ocean, 
                                                                 'geolat_t': geolat_t, 
                                                                 'geolon_t': geolon_t})
aice_m_coords

Now when the first time slice is plotted there are informative axes as there are now CF compliant dimensions `latitude` and `longitude`.

In [ ]:
aice_m_coords.isel(time=0).plot(aspect=2, size=10, cmap=cmap);

## Selection using `sel` and `slice`

Also selection using slices of latitude and longitude are easy using these new dimensions. e.g. selecting the Weddell Sea in Antarctica

In [ ]:
aice_m_coords.isel(time=0).sel(longitude=slice(-80, 0), latitude=slice(None, -50)).plot(size=10, cmap=cmap);

The same data can also be plotted and projected using `cartopy` which works fine using the 1D `latitude` and `longitude` coordinates as these are correct for all values south of 65N, so all of the Southern Hemisphere.

In [ ]:
fig = plt.figure(figsize=(14, 8))
ax = plt.axes(projection=ccrs.Orthographic(-40, -90))
ax.coastlines()
ax.gridlines()
aice_m_coords.isel(time=0).sel(longitude=slice(-80, 0),
                               latitude=slice(None, -50)).plot(ax=ax,
                                                               transform=ccrs.PlateCarree(),
                                                               cmap=cmap);

Simple `sel` with `slice` can also be used to select areas in the northern hemisphere, but they appear distorted  as this area is in the tripole, so the 1D latitude and longitude variables are not correct

In [ ]:
aice_m_coords.isel(time=0).sel(longitude=slice(-80, 80), 
                               latitude=slice(50, 85)).plot(size=8, aspect=2, cmap=cmap);

If the same selection is plotted in an Orthographic projection with `cartopy` it is obvious that land/sea boundaries are not correct, and artefacts from the tripole are visible

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax = plt.axes(projection=ccrs.Orthographic(0, 90))
ax.coastlines()
ax.gridlines()
aice_m_coords.isel(time=0).sel(longitude=slice(-80, 80), 
                               latitude=slice(50, 85)).plot(ax=ax,
                                                            cmap=cmap,
                                                            transform=ccrs.PlateCarree());

Using the true curvilinear coordinates to plot, the artefacts are removed, and the land/sea registration is correct, but the simple `sel` using the 1D coordinates means there are not constant lines of latitude or longitude in the selected data.

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax = plt.axes(projection=ccrs.Orthographic(0,90))
ax.coastlines()
ax.gridlines()
aice_m_coords.isel(time=0).sel(longitude=slice(-80, 80), 
                               latitude=slice(50, 85)).plot(ax=ax, cmap=cmap, 
                                                            transform=ccrs.PlateCarree(), 
                                                            x='geolon_t', y='geolat_t');

## Selection using `where`

To do proper spatial selection with 2D coordinates in the tripole area it is necessary to use the `xarray` [`where`](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.where.html) method, which will mask out the data where the stipulated condition is not met. With `drop=True` coordinate labels with only `False` values are removed.

In [ ]:
aice_m_coords.isel(time=0).where((geolon_t >= -80) & 
                                 (geolon_t <= 80) & 
                                 (geolat_t >= 50) & 
                                 (geolat_t <= 85), drop=True).plot(size=8, aspect=2, 
                                                                   x='geolon_t',
                                                                   y='geolat_t', 
                                                                   cmap=cmap)
plt.xlim([-80, 80])
plt.ylim([50, 85]);

Now when plotted in Orthographic projection with `cartopy` the selection is clearly along lines of constant latitude, and the left hand side selection is a constant longitude.

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax = plt.axes(projection=ccrs.Orthographic(0, 90))
ax.coastlines()
ax.gridlines()
aice_m_coords.isel(time=0).where((aice_m_coords.geolon_t >= -80) & 
                                 (aice_m_coords.geolon_t <= 80) & 
                                 (aice_m_coords.geolat_t >= 50) & 
                                 (aice_m_coords.geolat_t <= 85), drop=True).plot(ax=ax, 
                                                                                 transform=ccrs.PlateCarree(),
                                                                                 x='geolon_t',
                                                                                 y='geolat_t', 
                                                                                 cmap = cmap);